In [30]:
from scikit_roughsets.rs_reduction import RoughSetsSelector
import numpy as np
import pandas as pd

In [31]:
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('data/Zoo.csv')
# print(df.isnull().sum())
# labelencoder=LabelEncoder()

# df['target'] = labelencoder.fit_transform(df['target'])

In [32]:
y_series = df['target']
y = y_series.to_numpy().reshape(-1, 1)
feature_columns = [ col for col in df.columns if col not in ['id','target']]
# for col in feature_columns:
#     df[col] = pd.qcut(df[col], q=5, labels=False, duplicates='drop')
# feature_columns = [ col for col in df.columns if col not in ['id','target','REGION-PIXEL-COUNT']]
# print(df[feature_columns].isnull().sum())

# for col in feature_columns:
#     # 获取该特征的众数
#     mode = df[col].mode()[0]
#     # 用该特征的众数填充空值
#     df[col].fillna(mode, inplace=True)

X = np.array(df[feature_columns])
y=np.array(y)
selector = RoughSetsSelector()
X_selected = selector.fit(X, y).transform(X)
print(X_selected.shape)

(101, 5)


In [33]:
common_columns = []
for i in range(X.shape[1]):
    for j in range(X_selected.shape[1]):
        if (X[:, i] == X_selected[:, j]).all():
            common_columns.append(i)
print(common_columns)

[3, 5, 8, 11, 12]


In [34]:
df_train = pd.read_csv('data/Zoo.csv')
original_features = []
for col_idx in common_columns:
    original_features.append(feature_columns[col_idx])

In [35]:
original_features

['milk', 'aquatic', 'backbone', 'fins', 'legs']

In [36]:
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold

df = pd.read_csv('data/Zoo.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id', 'target']]
Y = df["target"]
X = df[original_features]
def SVM(trainX, trainY, testX, testY):
    scaler = StandardScaler()
    scaler.fit(trainX)
    trainX3 = scaler.transform(trainX)
    testX3 = scaler.transform(testX)
    clf = svm.SVC(C=10, gamma=0.8, max_iter=1000)
    clf.fit(trainX3, trainY)
    test_result = clf.predict(testX3)
    precision = np.sum(test_result == testY) / testY.shape[0]
    return precision

        # Use stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = []
for train_index, test_index in skf.split(X, Y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
    score = SVM(trainX, trainY, testX, testY)
    scores.append(score)
avg_score = np.mean(scores)
print(avg_score)

0.9800000000000001


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  UserWarning,


In [37]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('data/Zoo.csv')
feature_columns = [ col for col in df.columns if col not in ['id']]
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
X = df[original_features]
Y = df['target']
dt = DecisionTreeClassifier(max_depth=5)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.97


In [38]:
from sklearn.neighbors import KNeighborsClassifier

k = 5
knn = KNeighborsClassifier(n_neighbors=k)
df = pd.read_csv('data/Zoo.csv')
for col in feature_columns:
    labelencoder=LabelEncoder()
    df[col] = labelencoder.fit_transform(df[col])
feature_columns = [col for col in df.columns if col not in ['id','target']]
Y=df["target"]
X=df[original_features]
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print(mean_accuracy)

0.860909090909091
